In [1]:
from pandas import read_csv, DataFrame
import pandas as pd
import os
import re #Both patterns and strings to be searched can be Unicode strings as well as 8-bit strings.
import math
import cobra
import cobra.test
from __future__ import print_function
from os.path import join
from cobra.io import write_sbml_model

In [ ]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
data_dir = "/Users/lizrad/Dev/iVnat"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
model=cobra.io.read_sbml_model(join(data_dir, "iVnat.xml"))
#model=cobra.io.read_legacy_sbml("C:\\Users\Asus\Documents\Vibrio_project_literature\GCF_001456255.1_rast_metabolic_model.SBML\GCF_001456255.1_rast_metabolic_model.xml")
model

In [ ]:
#skip
#data_dir = "/Users/lizrad/Dev/iVnat"
#print("files found: ")
#print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
#model=cobra.io.read_sbml_model(join(data_dir, "iVnat_mapped.xml"))
#model=cobra.io.read_legacy_sbml("C:\\Users\Asus\Documents\Vibrio_project_literature\GCF_001456255.1_rast_metabolic_model.SBML\GCF_001456255.1_rast_metabolic_model.xml")
#model




### Reading the reac_xref

In [ ]:
reac_xref = read_csv("/Users/lizrad/Documents/Vibrio_folder/reac_xref.tsv" , sep='\t')
reac_xref

### Number of MNXR reactions in the model

In [ ]:
print ('This is the total amount of reactions in the model: %i' % (len(model.reactions)))

print ('We focus on mapping those %i reactions in the model that have an MNXR ID' % (len([met for met in model.reactions if met.id.startswith('MNXR') ])))

for met in model.reactions:
    if not met.id.startswith('MNXR'):
        print(met.id)

In [ ]:
bigg_reactions = [rxn for rxn in model.reactions if rxn.id.startswith('MNXR')]

print (len(bigg_reactions))
print (len(model.reactions))

In [ ]:
#skip
rxn_mnx2bigg = []
rxn_nobiggid = []

for rxn in [rxn for rxn in model.reactions if rxn.id.startswith('MNXR')]:
    if 'bigg' in rxn.annotation:
        reaction_id = rxn.annotation['BIGG'][0].split(', ')
        tag_split = rxn.id.split('_')
        rxn.id = str(reaction_id[0]) +'_'+ tag_split[1]
        rxn_mnx2bigg.append(reaction_id)
        cameo_model.repair()
    else:
        rxn_nobiggid.append(rxn)
        
print len(rxn_mnx2bigg)
print len(rxn_nobiggid)

### Formatting and creating Bigg MNX dictionary

In [ ]:

df_1 = reac_xref.loc[:,'XREF':'MNX_ID']
groups = df_1.groupby(['MNX_ID','XREF'])
df_1 = groups.apply(lambda x:list(x['XREF_ID']))
df_1 =df_1.unstack('XREF') 
reac_xref_MNX_1_dict = df_1.to_dict('index')
reac_xref_MNX_1_dict

In [ ]:

reac_property_MNX = read_csv("/Users/lizrad/Documents/Vibrio_folder/reac_xref.tsv" , sep='\t')
reac_property_MNX.set_index('MNX_ID',inplace = True)
df_3 = reac_property_MNX.fillna('MISSING')  #Fill NA/NaN values using the specified method
df_3
reac_property_MNX_dict = df_3.to_dict('index')
reac_property_MNX_dict

In [ ]:
#skip
#rxn_ID_not_in_MetaNetX_2 = [rxn for rxn in rxn_nobiggid if rxn.id.startswith('MNXR') and ''.join(re.findall('(MNXR\d*)_',rxn.id)) not in reac_xref_MNX_2_dict.keys()]
#print len(rxn_ID_not_in_MetaNetX_2)

In [ ]:
# Get a mapping from KEGG to the new MNXR IDs, for those reactions that cannot be found in the MNX Database,
# but that have a KEGG ID in their annotation. 
#153644- 170637
df_2 = reac_xref.loc[6:26957,'XREF_ID':'MNX_ID']
df_2 = df_2.set_index(['XREF_ID'])
seed_2_mnx_dict_2 = df_2.to_dict('index')
seed_2_mnx_dict_2

In [ ]:
reac_xref_MNXR_bigg = read_csv("/Users/lizrad/Documents/Vibrio_folder/reac_xref.tsv" , sep='\t', low_memory=False)
df_bigg = reac_xref_MNXR_bigg.loc[4:26955,'XREF':'MNX_ID']
pivoted_bigg = df_bigg.pivot_table(index='XREF',columns='XREF_ID',values='MNX_ID',aggfunc = lambda x: x)
pivoted_bigg=pivoted_bigg.drop(['metanetx'])
pivoted_bigg
reac_xref_MNX_bigg_dict = pivoted_bigg.to_dict('index')
reac_xref_MNX_bigg_dict2 = reac_xref_MNX_bigg_dict['bigg']
reac_xref_MNX_bigg_dict2

In [ ]:
not_in_reac_xref = []
in_reac_xref = []

for met in model.reactions:
    split=met.id.split("_")
    met_clean=split[0]
    print(met_clean)
    if not met_clean == '':
        
        if not met_clean in reac_xref_MNX_bigg_dict2.keys():
            not_in_reac_xref.append(met)
        else:
            in_reac_xref.append(met)

print ('%i metabolites with MNXR ID cannot be found in the MNX database' % (len(not_in_reac_xref)))
in_reac_xref

### Bigg Ids to reaction annotation

In [ ]:
for reac in model.reactions:
    reac.annotation

In [ ]:
for reac in model.reactions:
    data=''
    if 'metanetx.reaction' in reac.annotation:
        data=reac.annotation['metanetx.reaction'] 
        
        for xref in reac_xref_MNX_bigg_dict2.keys():
            if reac_xref_MNX_bigg_dict2[xref]==data:
                reac.annotation['bigg.reaction']= xref
                print(reac.annotation)
                
                


### Check for multiple ids 

In [ ]:
dupl_bigg = {}
for reac in model.reactions:
    if "bigg.reaction" not in reac.annotation.keys():  #skip EX reactions
        #print(reac.id)
        continue
    if reac.annotation["bigg.reaction"] not in dupl_bigg.keys():
        dupl_bigg.setdefault(reac.annotation["bigg.reaction"],[reac.id]) #similar to get(), but will set dict[key]=default if key is not already in dict.


    else:
        dupl_bigg[reac.annotation["bigg.reaction"]].append(reac.id)
        
dupl_bigg


In [ ]:
for ls in dupl_bigg.values():
    if len(ls) > 1:
        print(ls)

In [ ]:
for ls in dupl_bigg.values():
    if len(ls) > 1:
        print(ls)
        x=[]
        y=[]
        for members in ls: 
            main_id, comp =members.split("_")
            x.append(main_id)
            y.append(comp)
        if not x[0]==x[1]:
            print(x,y)
            for pair in zip(x,y):
                print(len(model.reactions.get_by_id(pair[0]+"_"+pair[1]).reaction))

In [ ]:
for reac in model.reactions:
    
    
    if "bigg.reaction" not in reac.annotation.keys():
        #print(reac.id)
        continue
    if reac.id.startswith('MNXR'):    
        #if reac.id in ["rxn03023_c0"]:
         #   pass
        #else:
        split, compartment=reac.id.split("_")
        reac.id
        reac.id=reac.annotation['bigg.reaction']+ '_' + compartment
        model.repair()
        print(reac.id)
    

In [ ]:
#chem_xref['#XREF'] = chem_xref['#XREF'].str.replace(':','\t')

#^[0-9A-Za-z]*:[0-9A-Za-z]*
#    (^[0-9A-Za-z]*):([0-9A-Za-z]*)
#        $1 \t $2
        



In [ ]:
write_sbml_model(model, "/Users/lizrad/Dev/iVnat/iVnat.xml")

In [ ]:
model.reactions.EX_cpd00001_e0.metabolites


In [ ]:
not_in_bigg=[]
for met in model.reactions:
    if "bigg.reaction" not in met.annotation:
        not_in_bigg.append(met.id)
len(not_in_bigg) 

In [ ]:
print(len(model.reactions))

In [ ]:
count_reac_more = 0
for reac in model.reactions:
    if "bigg.reaction"  in reac.annotation.keys():
        if type(reac.annotation["bigg.reaction"]) == list:
            count_reac_more+=1
count_reac_more

In [2]:
data_dir = "/Users/lizrad/Documents/Vibrio_folder"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
model_new=cobra.io.read_sbml_model(join(data_dir, "iVnat_newbigg.xml"))
#model=cobra.io.read_legacy_sbml("C:\\Users\Asus\Documents\Vibrio_project_literature\GCF_001456255.1_rast_metabolic_model.SBML\GCF_001456255.1_rast_metabolic_model.xml")
model_new

files found: 
iVnat_newbigg.xml, iVnatReport.html, iVnat_bigg.xml, iVnat_newbigg.json, iVnat_gapfill_glu.xml, iVnat_20180817.xml


Name,GCF_001456255_1_rast_metabolic_model
Memory address,0x0110e51198
Number of metabolites,1842
Number of reactions,1906
Objective expression,-1.0*bio1_reverse_b18f7 + 1.0*bio1
Compartments,"Cytosol_0, Extracellular_0"


In [8]:
with_bigg=0
for reac in model_new.reactions:
    print(reac.id)
    
    if "bigg.reaction" not in reac.annotation.keys():
        continue
    if reac.id.startswith("MNXR") or  reac.id==model_new.reactions.bio1:
        continue
       
    else:
        split, compartment=reac.id.rsplit("_", 1)
        with_bigg+=1
        if type(reac.annotation["bigg.reaction"]) == list:
            print("whole annotation:" ,reac.annotation['bigg.reaction'])
            
            reac.id=reac.annotation['bigg.reaction'][0]+ '_' + compartment
        else: 
            reac.id=reac.annotation['bigg.reaction']+ '_' + compartment
   
    print("new id :",reac.id)
            
    #if reac.id.startswith('cpd'):
model_new.repair()

       # print(reac.id)

DHPS2_c0
whole annotation: ['DHPS2', 'DHPS2_1', 'FOLD3', 'FOLD3_1', 'FOLD3m']
new id : DHPS2_c0
GTHS_c0
whole annotation: ['GTHS', 'GTHS_1']
new id : GTHS_c0
MNXR101695_c0
HXPRT_c0
new id : HXPRT_c0
ALCD21_D_c0
whole annotation: ['ALCD21_D', 'LCARR']
new id : ALCD21_D_c0
ASPARTOACYLASE-RXN_c0
MNXR96343_c0
ADNt2_c0
whole annotation: ['ADNt2', 'ADNt2pp']
new id : ADNt2_c0
ATPM_c0
whole annotation: ['ATPM', 'ATPPHm', 'ATPS', 'NTP1']
new id : ATPM_c0
GAMMA-GLUTAMYLCYCLOTRANSFERASE-RXN_c0
MNXR108115_c0
SAT_c0
whole annotation: ['SAT', 'SATm', 'SERAT']
new id : SAT_c0
CYTK1_c0
whole annotation: ['CYTK1', 'CYTK1m', 'CYTK1n']
new id : CYTK1_c0
MNXR99713_c0
MNXR105037_c0
ICHORT_c0
new id : ICHORT_c0
MNXR134818_c0
GCL_c0
whole annotation: ['GCL', 'GLUCYS']
new id : GCL_c0
MNXR97508_c0
NDPK7_c0
whole annotation: ['NDPK7', 'NDPK7m', 'NDPK7n']
new id : NDPK7_c0
PPCK_c0
whole annotation: ['PPCK', 'PPCKm']
new id : PPCK_c0
MNXR101984_c0
4-HYDROXYPROLINE-EPIMERASE-RXN_c0
MNXR102429_c0
MNXR136307_c0
OR

new id : NTD11_c0
ARYLDIALKYL-PHOSPHATASE-RXN_c0
MGSA_c0
new id : MGSA_c0
NTPTP1_c0
new id : NTPTP1_c0
MNXR137140_c0
MI3PP_c0
new id : MI3PP_c0
MNXR100469_c0
CHORS_c0
whole annotation: ['CHORS', 'CHRS']
new id : CHORS_c0
RXN-12448_c0
MNXR95315_c0
MNXR110643_c0
RXN0-5398_c0
MNXR108209_c0
MNXR96105_c0
MNXR134819_c0
MNXR95253_c0
MNXR109410_c0
UDPG4E_c0
new id : UDPG4E_c0
HKNDDH_c0
whole annotation: ['HKNDDH', 'HKNDDH_1']
new id : HKNDDH_c0
2.4.1.223-RXN_c0
MNXR101748_c0
RXN-5901_c0
MNXR105350_c0
ALAt2pp_c0
whole annotation: ['ALAt2pp', 'ALAt2r', 'ALAt2rL', 'ALAtm', 'ALAtx']
new id : ALAt2pp_c0
HAMR_c0
new id : HAMR_c0
TYRTA_c0
whole annotation: ['TYRTA', 'TYRTAh', 'TYRTAi', 'TYRTAim', 'TYRTAip', 'TYRTAm', 'TYRTAp']
new id : TYRTA_c0
AP4AH_c0
whole annotation: ['AP4AH', 'BTPS']
new id : AP4AH_c0
GLUCOSE-16-BISPHOSPHATE-SYNTHASE-RXN_c0
MNXR136820_c0
MNXR108883_c0
MNXR137158_c0
MNXR102410_c0
LYSabc_c0
whole annotation: ['LYSabc', 'LYSabcpp']
new id : LYSabc_c0
ACBIPGT_c0
new id : ACBIPGT_c0


whole annotation: ['BCTA_glu_h', 'LEUTA', 'LEUTAi', 'LEUTAm']
new id : BCTA_glu_h_c0
MNXR111160_c0
RXN-9663_c0
RIBabc_c0
whole annotation: ['RIBabc', 'RIBabcpp']
new id : RIBabc_c0
NAPHTHOATE-SYN-RXN_c0
ABUTD_c0
whole annotation: ['ABUTD', 'ABUTDm']
new id : ABUTD_c0
MNXR96325_c0
MNXR111858_c0
MNXR100748_c0
FACOAL180_c0
new id : FACOAL180_c0
RXN-9662_c0
VICIANIN-BETA-GLUCOSIDASE-RXN_c0
3.1.3.14-RXN_c0
D-LYSOPINE-DEHYDROGENASE-RXN_c0
MNXR97472_c0
MG2t_c0
whole annotation: ['MG2t', 'MG2tex', 'MG2th', 'MG2tpp', 'MG2tu', 'MGt5']
new id : MG2t_c0
MNXR135849_c0
NTD6_c0
whole annotation: ['NTD6', 'NTD6l', 'NTD6pp']
new id : NTD6_c0
DPR_c0
whole annotation: ['DPR', 'DPRm', 'PNORm', 'PNORn']
new id : DPR_c0
MNXR104466_c0
NADK_c0
whole annotation: ['NADK', 'NADKm']
new id : NADK_c0
MNXR95387_c0
LYSOLECITHIN-ACYLMUTASE-RXN_c0
MNXR99841_c0
NNDMBRT_c0
new id : NNDMBRT_c0
MNXR111060_c0
MNXR136213_c0
PUNP4_c0
new id : PUNP4_c0
MNXR136305_c0
MNXR134817_c0
MNXR107552_c0
SHCHF_c0
whole annotation: ['SHC

In [9]:
for reac in model_new.reactions:
    print(reac.id)

DHPS2_c0
GTHS_c0
MNXR101695_c0
HXPRT_c0
ALCD21_D_c0
ASPARTOACYLASE-RXN_c0
MNXR96343_c0
ADNt2_c0
ATPM_c0
GAMMA-GLUTAMYLCYCLOTRANSFERASE-RXN_c0
MNXR108115_c0
SAT_c0
CYTK1_c0
MNXR99713_c0
MNXR105037_c0
ICHORT_c0
MNXR134818_c0
GCL_c0
MNXR97508_c0
NDPK7_c0
PPCK_c0
MNXR101984_c0
4-HYDROXYPROLINE-EPIMERASE-RXN_c0
MNXR102429_c0
MNXR136307_c0
ORNabc_c0
MALT_c0
MNXR95384_c0
MNXR126081_c0
TDPDRE_c0
HACD7_c0
FLAVONE-APIOSYLTRANSFERASE-RXN_c0
SS-BUTANEDIOL-DEHYDROGENASE-RXN_c0
MNXR136769_c0
LYSDC_c0
ABTDG_c0
MNXR134793_c0
RXN3O-8214_c0
MNXR106405_c0
MNXR110225_c0
MNXR137716_c0
MNXR100753_c0
MNXR134889_c0
MNXR109713_c0
MNXR97378_c0
r0788_c0
PDXJ-RXN_c0
r0082_c0
PUNP5_c0
MNXR136227_c0
SOTA_c0
BETALDHx_c0
RXN-9888_c0
MNXR136266_c0
RXN0-6987_c0
OPHELINE-KINASE-RXN_c0
MNXR104241_c0
CYTK2_c0
RHCCE_c0
RXN-12481_c0
MNXR106487_c0
ADSL1r_c0
MNXR97200_c0
PPNCL2_c0
UDPGD_c0
GLU5K_c0
MNXR107108_c0
ANTHRANILATE-N-MALONYLTRANSFERASE-RXN_c0
ACGS_c0
NTD3_c0
MNXR134864_c0
RXN-11050_c0
MNXR109210_c0
MNXR102012_c0
MNX

MNXR104038_c0
PLASMALOGEN-SYNTHASE-RXN_c0
MICITDr_c0
SADH_c0
RXN-9539_c0
CBMKr_c0
ECOAH6_c0
NTD1_c0
MNXR137141_c0
2-OCTAPRENYLPHENOL-HYDROX-RXN_c0
MNXR95071_c0
MNXR134911_c0
KHte_c0
25-DIOXOPIPERAZINE-HYDROLASE-RXN_c0
MNXR96094_c0
MNXR134881_c0
MNXR99903_c0
2.7.7.34-RXN_c0
G5DHx_c0
MNXR102198_c0
PUNP7_c0
MNXR108918_c0
GLUS_nadph_c0
MNXR103420_c0
UMPK_c0
R524-RXN_c0
URIK1_c0
MNXR101698_c0
MNXR111785_c0
HIUH_c0
MNXR110584_c0
MNXR135827_c0
MNXR96326_c0
MNXR96758_c0
O2t_c0
RXN-8999_c0
GALACTONOLACTONASE-RXN_c0
FPPSh_c0
MNXR125940_c0
2.7.4.19-RXN_c0
HSST_c0
MNXR100254_c0
CHLt2_c0
MNXR97206_c0
ACONTa_c0
MNXR108757_c0
3HPPPNH_c0
RXN3DJ-170_c0
PHEPC_c0
UAMAGS_c0
MNXR135018_c0
MNXR109459_c0
MNXR135984_c0
MNXR102373_c0
GUAD_c0
MNXR97823_c0
RXN-10972_c0
RXN-4305_c0
MNXR134274_c0
DKDID_c0
INSK_c0
MNXR110579_c0
UPPDC1_c0
MNXR136199_c0
DDPGALA_c0
NDPK3_c0
SUCBZL_c0
MNXR111675_c0
3OXCOAT_c0
GALACTURONOKINASE-RXN_c0
MNXR97468_c0
ICHORS_c0
MNXR135883_c0
TMK_c0
MNXR105188_c0
APNPT_c0
PPENTOMUT-RXN_c0
MN

In [10]:
write_sbml_model(model_new, "/Users/lizrad/Documents/Vibrio_folder/iVnat_bigg.xml")